In [2]:
import fiftyone as fo
import fiftyone.types as fot
from pathlib import Path
import ty_fo_tools as ty

datasets = fo.list_datasets()
for dataset_name in datasets:
    print(f"\"{dataset_name}\",")

"my-dataset",
"sahi_null_v2_ms1_0605-0621_40_ok",
"sahi_null_v2_ms1_0710-0726_36_ok",
"sahi_null_v2_ms1_0726-0809_11_ok",
"sahi_null_v2_ms1_0809-0823_34_ok",
"sahi_null_v2_ms2_0726-0809_13_ok",
"sahi_null_v2_ms2_0809-0823_10_ok",
"sahi_null_v2_sw1_0605-0613_07_ok",
"sahi_v1_ms1_0605-0621_40_ok",
"sahi_v1_ms1_0710-0726_36_ok",
"sahi_v1_ms1_0726-0809_11_ok",
"sahi_v1_ms1_0809-0823_34_ok",
"sahi_v1_ms2_0726-0809_13_ok",
"sahi_v1_ms2_0809-0823_10_ok",
"sahi_v1_sw1_0605-0613_07_ok",
"v1_ms1_0710-0726_36_ok",
"v1_ms1_0809-0823_34_ok",
"v1_ms2_0726-0809_13_ok",
"v1_sw1_0605-0613_07_ok",
"v2_ms1_0605-0621_40_ok",
"v2_ms1_0726-0809_11_ok",
"v2_ms2_0809-0823_10_ok",
"v3_sahi_ms1_0605-0621_40_ok",
"v3_sahi_ms1_0710-0726_36_ok",
"v3_sahi_ms1_0726-0809_11_ok",
"v3_sahi_ms1_0809-0823_34_ok",
"v3_sahi_ms2_0726-0809_13_ok",
"v3_sahi_ms2_0809-0823_10_ok",
"v3_sahi_sw1_0605-0613_07_ok",


In [3]:

dataset_name = "sahi_null_v2_ms1_0605-0621_40_ok"
dataset = fo.load_dataset(dataset_name)
dataset

Name:        sahi_null_v2_ms1_0605-0621_40_ok
Media type:  image
Num samples: 243
Persistent:  True
Tags:        []
Sample fields:
    id:                                                                           fiftyone.core.fields.ObjectIdField
    filepath:                                                                     fiftyone.core.fields.StringField
    tags:                                                                         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                                                                     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:                                                                   fiftyone.core.fields.DateTimeField
    last_modified_at:                                                             fiftyone.core.fields.DateTimeField
    ground_truth:                                                                 fiftyone.co

### 导出evaluation结果

In [5]:
import fiftyone as fo
import pandas as pd
import numpy as np


def get_evaluation_data_like_gui(dataset, eval_key):
    """模拟GUI中的数据获取方式"""
    results = dataset.load_evaluation_results(eval_key)
    info = dataset.get_evaluation_info(eval_key)

    # 获取基础指标
    metrics = results.metrics()

    # 计算平均置信度（GUI中的逻辑）
    avg_confs = {}
    if hasattr(results, "confs") and hasattr(results, "ypred"):
        counts = {}
        sums = {}
        for yp, conf in zip(results.ypred, results.confs):
            if yp not in counts:
                counts[yp] = 0
                sums[yp] = 0.0
            counts[yp] += 1
            sums[yp] += conf if conf is not None else 0.0

        for c in results.classes:
            avg_confs[c] = sums[c] / counts[c] if counts.get(c, 0) > 0 else 0.0

    # 计算总体平均置信度
    if avg_confs:
        metrics["average_confidence"] = np.mean(list(avg_confs.values()))

    # 获取TP/FP/FN
    tp = (
        dataset.sum(f"{eval_key}_tp")
        if f"{eval_key}_tp" in dataset.get_field_schema()
        else 0
    )
    fp = (
        dataset.sum(f"{eval_key}_fp")
        if f"{eval_key}_fp" in dataset.get_field_schema()
        else 0
    )
    fn = (
        dataset.sum(f"{eval_key}_fn")
        if f"{eval_key}_fn" in dataset.get_field_schema()
        else 0
    )

    metrics["tp"] = tp
    metrics["fp"] = fp
    metrics["fn"] = fn

    # 尝试获取mAP/mAR
    try:
        metrics["map"] = results.mAP()
    except:
        metrics["map"] = None

    try:
        metrics["mar"] = results.mAR()
    except:
        metrics["mar"] = None

    return metrics, info


# 使用示例
# dataset = fo.load_dataset("sahi_null_v2_ms1_0605-0621_40_ok")
datasets_list = [
    "sahi_null_v2_ms1_0605-0621_40_ok",
    "sahi_null_v2_ms1_0710-0726_36_ok",
    "sahi_null_v2_ms1_0726-0809_11_ok",
    "sahi_null_v2_ms1_0809-0823_34_ok",
    "sahi_null_v2_ms2_0726-0809_13_ok",
    "sahi_null_v2_ms2_0809-0823_10_ok",
    "sahi_null_v2_sw1_0605-0613_07_ok",
]

for dataset_name in datasets_list:
    print(f"Processing dataset: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    eval_keys = dataset.list_evaluations()

    all_data = []
    for eval_key in eval_keys:
        try:
            metrics, info = get_evaluation_data_like_gui(dataset, eval_key)

            row = {
                "eval_key": eval_key,
                "eval_type": info.config.type,
                "eval_method": info.config.method,
                "average_confidence": metrics.get("average_confidence", 0),
                "support": metrics.get("support", 0),
                "accuracy": metrics.get("accuracy", 0),
                "iou_threshold": getattr(info.config, "iou", 0.5),
                "precision": metrics.get("precision", 0),
                "recall": metrics.get("recall", 0),
                "f1_score": metrics.get("fscore", 0),
                "map": metrics.get("map"),
                "mar": metrics.get("mar"),
                "true_positives": metrics.get("tp", 0),
                "false_positives": metrics.get("fp", 0),
                "false_negatives": metrics.get("fn", 0),
            }
            all_data.append(row)

        except Exception as e:
            print(f"处理 {eval_key} 时出错: {e}")

    df = pd.DataFrame(all_data)
    df.to_excel(f"evaluation_{dataset_name}.xlsx", index=False)

Processing dataset: sahi_null_v2_ms1_0605-0621_40_ok
Processing dataset: sahi_null_v2_ms1_0710-0726_36_ok
Processing dataset: sahi_null_v2_ms1_0726-0809_11_ok
Processing dataset: sahi_null_v2_ms1_0809-0823_34_ok
Processing dataset: sahi_null_v2_ms2_0726-0809_13_ok
Processing dataset: sahi_null_v2_ms2_0809-0823_10_ok
Processing dataset: sahi_null_v2_sw1_0605-0613_07_ok


### 替换原有评估的方法，更加完整。有mAP

In [ ]:
import fiftyone as fo
from tqdm import tqdm

datasets_list = [
    # "sahi_null_v2_ms1_0605-0621_40_ok",
    "sahi_null_v2_ms1_0710-0726_36_ok",
    "sahi_null_v2_ms1_0726-0809_11_ok",
    "sahi_null_v2_ms1_0809-0823_34_ok",
    "sahi_null_v2_ms2_0726-0809_13_ok",
    "sahi_null_v2_ms2_0809-0823_10_ok",
    "sahi_null_v2_sw1_0605-0613_07_ok",
]

for dataset_name in datasets_list:
    print(f"\n📦 处理数据集: {dataset_name}")
    dataset = fo.load_dataset(dataset_name)

    eval_keys = dataset.list_evaluations()

    # =========================
    # 1️⃣ 创建所有新评估（带进度条）
    # =========================
    new_evaluations = {}

    for eval_key in tqdm(
        eval_keys,
        desc="创建新评估",
        leave=False,
    ):
        try:
            eval_info = dataset.get_evaluation_info(eval_key)

            dataset.evaluate_detections(
                eval_info.config.pred_field,
                gt_field=eval_info.config.gt_field,
                eval_key=f"{eval_key}_new",
                compute_mAP=True,
                method=eval_info.config.method,
                iou=eval_info.config.iou,
                classwise=eval_info.config.classwise,
            )

            new_evaluations[eval_key] = f"{eval_key}_new"

        except Exception as e:
            tqdm.write(f"✗ 创建 {eval_key}_new 失败: {e}")

    # =========================
    # 2️⃣ 替换原评估（带进度条）
    # =========================
    for old_key, new_key in tqdm(
        new_evaluations.items(),
        desc="替换评估",
        leave=False,
    ):
        try:
            dataset.delete_evaluation(old_key)
            dataset.rename_evaluation(new_key, old_key)

        except Exception as e:
            tqdm.write(f"✗ 替换 {old_key} 失败: {e}")

    print(f"✅ 数据集完成: {dataset_name}")
